In [1]:
import numpy as np
import os

In [2]:
#finds difference in number of days between two dates
#helper function for month_days
def month_days(month,year):
    days={1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    if year%4==0 or year%400==0 :
        days[2]+=1
    return days[month]

In [3]:
#finding number of days between 2 dates
def day_diff(dstart_date,dend_date):
    start_date=dstart_date[:]
    end_date=dend_date[:]
    days=0
    while(end_date[0]-start_date[0]>1):
        if(start_date[0]%4==0 | start_date[0]%400==0):
            days+=366
        else:
            days+=365
        start_date[0]+=1
    if(end_date[0]>start_date[0]):
        days+=month_days(start_date[1],start_date[0])-start_date[2]+1
        start_date[1]+=1
        start_date[2]=1
        while(start_date[1]<=12):
            days+=month_days(start_date[1],start_date[0])
            start_date[1]+=1
        start_date[1]=1
        start_date[0]+=1
    if(end_date[1]>start_date[1]):
        days+=month_days(start_date[1],start_date[0])-start_date[2]+1
        start_date[1]+=1
        start_date[2]=1
    while(end_date[1]-start_date[1]>1):
        days+=month_days(start_date[1],start_date[0])
        start_date[1]+=1
    if(end_date[1]>start_date[1]):
        days+=month_days(start_date[1],start_date[0])-start_date[2]+1
        start_date[1]+=1
        start_date[2]=1
    days+=end_date[2]-start_date[2]
    return days
print(day_diff([2020,1,10],[2020,1,29]))   

19


In [4]:
#incrementing a date by a number of days
def day_inc(start_date,days):
    m_days_left=month_days(start_date[1],start_date[0])-start_date[2]
    if m_days_left<days:
        start_date[1]+=1
        start_date[2]=1
        days-=m_days_left+1
        if(start_date[1]>12):
            start_date[0]+=1
            start_date[1]=1
        return day_inc(start_date,days)
    else:
        start_date[2]+=days
        days=0
    #final date or end date is returned below
    return start_date
day_inc([2019,12,31],60)

[2020, 2, 29]

In [5]:
#creates symptoms for patients from a predefined dataset
#helper function for result_generator
def symptom_generator():
    symptoms=['cough','fever','sore_throat','pneumonia','diarheaa','fatigue','muscle_pain','shivering','dizziness']
    l=len(symptoms)
    prob=np.random.randint(5,size=l)
    arr=['discomfort']
    for i in range(l):
        if(prob[i]>2):
            arr.append(symptoms[i])
    return arr

In [6]:
#generates result of diagnosis with number of days, basically creates a patient
#then creates their symptoms, based on severity of symptoms creates a probability based death or survival
#flag=1-dead,0-healed,-1-unresolved
def result_generator(start_date,datecap):
    symptoms=symptom_generator()
    severity={'discomfort':0.0,'cough':0.001,'fever':0.001,'sore_throat':0.0005,'pneumonia':0.055,'diarheaa':0.005,'fatigue':0.005,'muscle_pain':0.005,'shivering':0.007,'dizziness':0.016}
    sevsum=0
    for sym in symptoms:
        sevsum+=severity[sym]
    persistence=sevsum*8
    healprob=(1-sevsum-persistence)
    #healingprobability in a day and severity sum must be quite low compared to persistence for disease to last long
    #hence the modifications below
    healprob=healprob/5.0
    sevsum=sevsum/5.0
    persistence=persistence/5.0+0.8
    flag=-1
    num=0
    while num<datecap:
        r=np.random.random()
        num+=1
        if(r<healprob):
            flag=0
            break
        elif(r>1-sevsum):
            flag=1
            break
    return [num-1,flag,symptoms]
result_generator([2019,12,31],5)

[4, -1, ['discomfort', 'fever', 'muscle_pain', 'shivering']]

In [7]:
#simulates a slightly random exponential rise in numbers based on an iter number
#the m_param here is equal to e^(ln(Ro)/I)
#where Ro is the reproduction number and I is the incubation period average.
#this m_param is the factor by which the cases grow daily
#Max_Cap is added here as regions with more population usually have a larger density.
#Hence higher rate of infection
def exp_pat_num_gen(iternum,m_param,Max_Cap):
    n=(m_param**iternum)
    return int(2*n/3.0+np.random.random()*2*n/3.0)
print(exp_pat_num_gen(7,1.1,200))

2


In [8]:
#generates patient list for a hospital with a hidden epidemic with provided Ro
def pat_gen_hospital(start_date,end_date,Ro,file,start_prob,day_delay,min_pat,max_pat):
    day_delay=day_diff(start_date,end_date)-day_delay-1
    start_param=100.0*(1-start_prob)
    days_run=day_diff(start_date,end_date)
    todate=start_date
    Max_cap=np.random.randint(min_pat,max_pat)
    #print("Capacity of hospital %d"%Max_cap)
    file.write("#Capacity of hospital %d\n"%Max_cap)
    flag=0
    i=0
    patient_arr=[]
    patient_num=1
    exp_count=0
    for i in range(days_run):
        daily_add=np.random.randint(1,1+Max_cap/8)
        #IMPRORTANT
        #set np.random.random()*100>100 if you dont want an epidemic
        #set np.random.random()*100>0 if you want the epidemic to start immediately at set date
        #set i>n days where n is a number of days to delay the start of the epidemic
        #make sure you adjust the start of the epidemic appropriately
        #can also be set at the function in the end
        if(flag==1):
            daily_add+=exp_pat_num_gen(exp_count,Ro,Max_cap)
            exp_count+=1
        elif flag==0 and np.random.random()*100>start_param and i>=day_delay:
            flag=1
        for j in range(daily_add):
            arr=[patient_num]+[todate]+ (result_generator(todate,days_run-i))
            #print("arr",arr)
            for k in range(4):
                if k==1:
                    #print("running")
                    file.write("%d-%d-%d "%(arr[1][0],arr[1][1],arr[1][2]))
                else:
                    file.write("%d "%arr[k])
            sym=arr[4]
            for s in sym:
                file.write("%s,"%s)
            file.write("\n")
            #patient_arr.append(arr)
            #print(patient_arr)
            patient_num+=1
        todate=day_inc(start_date,1)
    #return patient_arr

In [9]:
#print(pat_gen_hospital([2020,1,10],[2020,1,13],2))

In [10]:
#makes num number of hospital entry between the start and end dates with a Ro and num is specified
def hospital_gen(start_date,end_date,Ro,num,start_prob,day_delay,min_pat,max_pat):
    if not os.path.exists('Hospitals'):
        os.makedirs('Hospitals')
    for i in range(num):
        todate=start_date[:]
        name=".\Hospitals\hospital_"+str(i+1)+".txt"
        fw=open(name,'w')
        pat_gen_hospital(todate,end_date,Ro,fw,start_prob,day_delay,min_pat,max_pat)
        fw.close()
    print("generated data")

In [11]:
#clears previously created hospital data
def remove_files():
    i=1
    while(os.path.exists("./Hospitals/hospital_%d.txt"%i)):
        os.remove("./Hospitals/hospital_%d.txt"%i)
        i+=1
remove_files()

In [15]:
#parameters are in the folowing order
#1)start date of data simulation
#2)end date
#3)growth parameter (not Ro)
#4)number of hospitals/taluks to be simulated
#5)probability of epidemic. set it to 1 if you want the epidemic for sure and 0 for no epidemic
#6)days before the end date to which we delay start of epidemic, after that it starts with the previous probability
#7)minimum capacity of each entity hospital or taluk
#8)maximum capacity
hospital_gen([2020,1,25],[2020,4,30],1.10,62,1.0,90,50,250)

generated data
